In [1]:
import nltk
import numpy as np

In [2]:
from nltk.corpus import words
word_list = words.words()
# prints 236736
print(len(word_list))

# or we can use more comomon words list first
from english_words import english_words_lower_alpha_set
word_list = english_words_lower_alpha_set
print(len(word_list))

236736
25463


In [3]:
'ghost' in word_list

True

In [4]:
len(word_list)

25463

In [5]:
# get 5 letter words from word_list
five_letter_words = [w for w in word_list if len(w) == 5]

In [6]:
len(five_letter_words)

3213

In [7]:
#select a random word from the list
import random
solution = random.choice(list(five_letter_words))
solution

'basal'

In [24]:
def play_game(solution = 'random'):
    # get a random word from the list
    if solution == 'random':
        solution = random.choice(list(five_letter_words))
        
    board = ["0"] * 5 * 6
    score_board = ["x"] * 5 * 6
    # print the board
    print(board)
    
    def play_one_chance():
        # get the player to enter a letter
        word_input = input("Enter a word: ")
        if word_input in five_letter_words:
            print("Five letter word found as input!")
            return(word_input)
        else:
            print("No five letter word found as input! Give something else!")
            return(play_one_chance())

    def update_board(board,i,word_input):
        board[(i * 5):(5 + i * 5)] = word_input
        return(board)

    def update_score_board(score_board,i,word_input):
        score_board[(i * 5):(5 + i * 5)] = ["g" if (word_input[w] in solution and  word_input[w]==solution[w]) else
                "y" if (word_input[w] in solution and word_input[w]!=solution[w]) else "b" for w in range(5)]
        return(score_board)

    for i in range(6):
        word_input = play_one_chance()
        if word_input == solution:
            # convert string to list
            word_input = list(word_input)

            board = update_board(board,i,word_input)
            score_board = update_score_board(score_board,i,word_input)

            print(board)
            print(score_board)
            print("Solution found!")

            player_won = True
            break
        else:
            print("Solution not found!")
            # convert string to list
            word_input = list(word_input)

            board = update_board(board,i,word_input)
            score_board = update_score_board(score_board,i,word_input)

            print(board) 
            print(score_board)
            player_won = False


    if player_won == False:
        print("Game over you lost!")
    if player_won == True:
        print("You won!")

    print("The solution was: " + solution)

    return



In [23]:
play_game('sound')

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
Five letter word found as input!
Solution not found!
['a', 'p', 'p', 'l', 'e', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['b', 'b', 'b', 'b', 'b', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']
Five letter word found as input!
Solution not found!
['a', 'p', 'p', 'l', 'e', 'p', 'h', 'o', 'n', 'e', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['b', 'b', 'b', 'b', 'b', 'b', 'b', 'y', 'g', 'b', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']
Five letter word found as input!
['a', 'p', 'p', 'l', 'e', 'p', 'h', 'o', 'n', 'e', 's', 'o', 'u', 'n', 'd', '0', '0', '0', '0', '0', '0', 